In [1]:
%pip install -qU langchain langgraph langsmith langchain_core langchain_community langchain_openai python-dotenv jupyterlab notebook

Note: you may need to restart the kernel to use updated packages.


In [8]:
#Exploring messages from langchain_core.messages AIMessage, HumanMessage, SystemMessage
from langchain_core.messages import AIMessage, SystemMessage, HumanMessage
from pprint import pprint

messages = [AIMessage(content="Hi!",name="model")]
messages.extend([SystemMessage(content="You are a usefull model to answer", name="System")])
messages.extend([HumanMessage(content="I am good", name="human")])

for m in messages:
    m.pretty_print()

================================== Ai Message ==================================
Name: model

Hi!
================================ System Message ================================
Name: System

You are a usefull model to answer
================================ Human Message =================================
Name: human

I am good


In [11]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
# Ctreating llm
from langchain.chat_models import init_chat_model
from langchain.tools import tool
llm = init_chat_model(model='gpt-4o',model_provider='openai')

#creating tools

@tool
def multiply(a:int, b:int):
    """ This function will calculate multiplication of given value
    Args:
         a : first input as typed int
         b : second input as typed int

    return: multipication of two numbers

    """
    return a * b

# initilize tool caling llm

llm_with_tools = llm.bind_tools([multiply])

messages = [SystemMessage(content="Hi! you are a helpfull llm please use the multiply tool if you ask for perform multiplication", name="System")]
messages.extend([HumanMessage(content="Please multiply 5 and 7", name="Human")])

responce = llm_with_tools.invoke(messages)
print(responce)

content='' additional_kwargs={'tool_calls': [{'id': 'call_cb9MuUZ2effcF5aG3pHzKZPV', 'function': {'arguments': '{"a":5,"b":7}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 109, 'total_tokens': 127, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_92f14e8683', 'id': 'chatcmpl-BLtlmhiD6B7zdFQ2QOkTp7kf49crB', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-2d12c302-34a8-44c0-a04e-d818f1b5407a-0' tool_calls=[{'name': 'multiply', 'args': {'a': 5, 'b': 7}, 'id': 'call_cb9MuUZ2effcF5aG3pHzKZPV', 'type': 'tool_call'}] usage_metadata={'input_tokens': 109, 'output_tokens': 18, 'total_tokens': 127, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio

In [26]:
#State with list of messages

from langchain_core.messages import AnyMessage
from typing import Annotated, TypedDict, List
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from IPython.display import display, Image

class State(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]

# As this thing frequenty used in langgraph we have prebuild class for the above ,

from langgraph.graph import MessagesState

class MessagesState(MessagesState):
    #messages key pre-installed
    pass

# define node

def call_tool_llm(state:MessagesState):
    return {"messages": llm_with_tools.invoke(state["messages"])}

builder = StateGraph(MessagesState)

builder.add_node("call_tool_llm", call_tool_llm)
builder.add_edge(START,"call_tool_llm")
builder.add_edge("call_tool_llm", END)

graph = builder.compile()

#display(Image(graph.get_graph().draw_mermaid_png()))

In [27]:
# 1. Creating LLM
from langchain.chat_models import init_chat_model
from langchain_core.messages import SystemMessage, HumanMessage, AnyMessage
from langchain.tools import tool
from typing import Annotated, TypedDict, List
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from IPython.display import display, Image
from langgraph.graph import MessagesState

# Initialize the chat model with tools support
llm = init_chat_model(model='gpt-4o', model_provider='openai')

# 2. Creating a tool
@tool
def multiply(a: int, b: int):
    """Multiply two integers."""
    return a * b

# 3. Bind the tool to the model
llm_with_tools = llm.bind_tools([multiply])

# 4. Prepare a basic test (optional, just to see it working)
messages = [
    SystemMessage(content="Hi! You are a helpful LLM. Please use the multiply tool if asked to perform multiplication."),
    HumanMessage(content="Please multiply 5 and 7")
]
response = llm_with_tools.invoke(messages)
print("Direct Response from tool-bound LLM:", response)

# 5. Define LangGraph State (Optional since MessagesState already exists)
# You do not need this class unless you’re customizing the state
# Keeping it to match your original structure, but it's optional
class CustomMessagesState(MessagesState):
    pass

# 6. Define a node function
def call_tool_llm(state: MessagesState):
    return {"messages": llm_with_tools.invoke(state["messages"])}

# 7. Build the graph
builder = StateGraph(MessagesState)
builder.add_node("call_tool_llm", call_tool_llm)
builder.add_edge(START, "call_tool_llm")
builder.add_edge("call_tool_llm", END)
graph = builder.compile()

print(graph.invoke({"messages":"multiplication of 10 and 5"}))
# # 8. Visualize the graph
#display(Image(graph.get_graph().draw_mermaid_png()))



Direct Response from tool-bound LLM: content='' additional_kwargs={'tool_calls': [{'id': 'call_Gp5hxMgKgV3yA2opzbAn1JIb', 'function': {'arguments': '{"a":5,"b":7}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 71, 'total_tokens': 89, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_432e014d75', 'id': 'chatcmpl-BLutJaZnAz2tTnWTT3U7LUFCix6R8', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-d48f2810-e555-4852-8c2a-644eba33a33c-0' tool_calls=[{'name': 'multiply', 'args': {'a': 5, 'b': 7}, 'id': 'call_Gp5hxMgKgV3yA2opzbAn1JIb', 'type': 'tool_call'}] usage_metadata={'input_tokens': 71, 'output_tokens': 18, 'total_tokens': 89, 'input_token_details': {'audio': 0, 'cache_read': 0}

In [6]:
from langchain.chat_models import init_chat_model
from langgraph.graph import START, StateGraph, END, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain.tools import tool

#llm
llm = init_chat_model(model="gpt-4o", model_provider='openai')

#build tools
@tool
def multiply(a:int, b:int):
    """Multiply two integers."""
    return a * b
@tool
def summ(a:int, b:int):
    """Sum or Add two integers."""
    return a + b

llm_with_tools = llm.bind_tools([multiply, sum])

def call_tool_llm(state:MessagesState):
    return {"messages":[llm_with_tools.invoke(state['messages'])]}

builder = StateGraph(MessagesState)

builder.add_node("call_tool_llm", call_tool_llm)
builder.add_node("tools",ToolNode([multiply]))
builder.add_edge(START, "call_tool_llm")
builder.add_conditional_edges("call_tool_llm",tools_condition)
builder.add_edge("tools", END)

graph = builder.compile()

messages = graph.invoke({"messages":"5+ 5 "})

for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

5+ 5 
================================== Ai Message ==================================

5 + 5 equals 10.
